In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Simulation of Conditioned Diffusions on Riemannian Manifolds

Mathias Højgaard Jensen and Stefan Sommer

# SO(3)

In [ ]:
%cd ..

In [ ]:
# SO(3)
from src.groups.SON import *
G = SON(3,invariance='right')
print(G)

from src.plotting import *

In [ ]:
from src.group import invariant_metric
invariant_metric.initialize(G)

from src.group import energy
energy.initialize(G)

In [ ]:
q = np.array([1e-6,0,0])
g = G.psif(q)
v = np.array([0,1,1])
p = G.sharppsif(q,v)

In [ ]:
# sample data
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

G.sigma.set_value(np.diag((1.,.3,1.4))) # set metric
K = 16 # 1024
obss = np.zeros((K,)+g.shape)
# srng.seed(422)
for i in range(K):
    (ts,gs) = G.Brownian_invf(g,dWsf(G.dim.eval()))
    obss[i] = gs[-1]

# plot samples
newfig()
for i in range(K):
    G.plotg(obss[i])
plt.show()

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# parameters
g0 = G.sym_element()
thetas = (g0, G.sigma,)

# guide function
# phi = lambda g,v: G.LAtoV(G.invtrns(G.inv(g),v)-G.e)
phi = lambda g,v: T.tensordot(G.inv(G.sigma),G.LAtoV(G.log(G.invtrns(G.inv(g),v))),(1,0))

(Brownian_inv_guided,Brownian_inv_guidedf) = get_guided_likelihood(
    G,G.sde_Brownian_inv,phi,lambda g: G.sigma,
    A=G.gG, integration='stratonovich')  

w = G.psif(v)
(ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,w,dWsf(G.dim.eval()))[:4]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

newfig()
G.plot_path(gs)
G.plotg(w,color='k')
# plt.savefig('/home/stefan/Dropbox/projects/diffusion/figures/SO3-bridge.pdf')
plt.show()

In [ ]:
n_steps.set_value(20)
options = {}
options['samples_per_obs'] = 2
options['epochs'] = 100
options['learning_rate'] = 5.e-3
options['varphi_update_rate'] = 1.
options['verbose'] = True
options['initial'] = [g, # random value
                      np.diag((.5,.5,.5)),]
# options['update_v'] = lambda g: theano.gradient.disconnected_grad(Brownian_inv_fiber(g,dWs(G.dim))[1][-1])
# options['update_vf'] = lambda g: Brownian_inv_fiberf(g,dWsf(G.dim.eval()))[1][-1]

In [ ]:
# Transition density
v0 = G.sym_element()
p_Tf = theano.function([g0,v0],p_T(g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma))
log_p_Tf = theano.function([g0,v0],log_p_T(g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma))
dlog_p_Tf = theano.function([g0,v0],dlog_p_T(thetas,g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma))

# G.sigma.set_value(np.diag((1.,.3,1.6))) # set metric

# # on G
%time print(p_Tf(g,G.psif(v))) 
%time print(log_p_Tf(g,G.psif(v))) 
%time print(dlog_p_Tf(g,G.psif(v)))

In [ ]:
# samples for MLE
# G.sigma.set_value(1.*np.eye(G.dim.eval())) # set metric, uniform
G.sigma.set_value(np.diag((.2,.2,.8))) # anisotropic
K = 128

thetas_true = [g]+[G.sigma.eval(),]

obss = np.zeros((K,)+g.shape)
# srng.seed(422)
for i in range(K):
    (ts,gs) = G.Brownian_invf(g,dWsf(G.dim.eval()))
    obss[i] = gs[-1]

# plot samples
newfig()
for i in range(K):
    G.plotg(obss[i])
plt.show()

In [ ]:
# on SO(3)
from src.statistics.mle import *

def llog_p_T(thetas,pars):
    (v,seed) = pars
    if seed:
        srng.seed(seed)
    g = thetas[0]
    G.sigma.set_value(thetas[1])
    return dlog_p_Tf(g,v)

def update_thetas(thetas, dthetas):
    g = thetas[0]
    sigma = thetas[1]
    
    #g = G.to_groupf(g+options['learning_rate']*dthetas[0])
    sigma += options['learning_rate']*dthetas[1]
    
    return (g,sigma)

# run MLE
(thetas, log_likelihood, log_likelihoods, thetass) = iterative_mle(obss,llog_p_T,update_thetas,options)

# plot
plt.plot(range(options['epochs'])[2:],log_likelihoods[2:])
plt.show()
plt.plot(range(options['epochs']),thetass[0].reshape((thetass[0].shape[0],-1)))
plt.hlines(thetas_true[0].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
plt.plot(range(options['epochs']),thetass[1].reshape((thetass[1].shape[0],-1)))
plt.hlines(thetas_true[1].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
None